In [1]:
# 라이브러리 설치

# !pip install -q langchain
# !pip install -q langchain-community
# !pip install -q tiktoken
# !pip install -q langchain_openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# env 

from dotenv import load_dotenv

# youtube-transcript-api
from langchain_community.document_loaders import YoutubeLoader
# 프롬프트 작성 템플릿 제공
from langchain_core.prompts import PromptTemplate
# 긴글요약
from langchain.chains.summarize import load_summarize_chain
# 텍스트 분리
from langchain.text_splitter import RecursiveCharacterTextSplitter
# OpenAI 
from langchain_openai import ChatOpenAI


load_dotenv()

True

In [ ]:
# 유튜브 자막 추출

loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=Pn-W41hC764")
transcript = loader.load()
print(transcript)

In [ ]:
# 자막을 쪼개서 요약 
# chunk_size : 각각 분할한 문서 사이즈

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
text = text_splitter.split_documents(transcript)

In [ ]:
# 프롬프트 템플릿 지정
prompt = PromptTemplate(
    template="Summerize the youtube video whose transcript is provided within backticks {text}",
    input_variables=['text'])

# 요약된 내용들을 취합하여 재요약
combine_prompt = PromptTemplate(
    template="Combine all the youtube video whose transcript provided within backticks {text} \
        Provide a concise summary between 8 to 10 sentences.",
    input_variables=['text'])

# 모델 지정
llm = ChatOpenAI(temperature=0, max_completion_tokens=3000, model='gpt-4o-mini',request_timeout=120)

# 요약 실행
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False,
                              map_prompt=prompt, combine_prompt=combine_prompt)
output = chain.invoke(text)

In [ ]:
# 번역(영어 => 한국)

from googletrans import Translator

translator = Translator()

result = await translator.translate(output["output_text"], dest="ko")
print(result.text)